In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from matplotlib.pylab import *
from skimage.io import imread
from glob import glob
from bunch import Bunch

In [ ]:
import caffe
from caffe import Net

In [ ]:
caffe.set_device(1)
caffe.set_mode_gpu()

**NOTE:** The 'deploy.prototxt' generated by fix_bn nees 'samlpe_weights_test:true' to be added to each dropout_param. 

This is after 115000 iterations

In [ ]:
net = Net('Models/Inference/deploy.prototxt', 'Models/Inference/test_weights.caffemodel', caffe.TEST)

Please refer to the console where the jupyter notebook is running. 
That will have a ridiculous amount of messages logged by caffe. 

In [ ]:
IGNORE_INDEX = 0
WINDOW_INDEX = 1
DOOR_INDEX = 2
FACADE_EDGE_INDEX = 3
OTHER_INDEX = 4

from collections import OrderedDict
COLORS = OrderedDict()
COLORS['Ignore'] = [0,0,0]
COLORS['Window'] = [51, 51, 255]
COLORS['Door'] = [51, 204, 51]
COLORS['FacadeEdge'] = [255,255,0]
COLORS['OtherFacade'] = [255,128,128]
INDEX = Bunch({k:COLORS.keys().index(k) for k in COLORS })

In [ ]:
ax = gca()
ylim(0, 28)
xlim(0, 12)
for i, (k,v) in enumerate(COLORS.items()):
    y = 26-2*i
    r = Rectangle((.5,y), 2, 1.5, color=array(v)/255.)
    text(2.8, y+0.5, k)
    text(0, y+0.5, str(i))
    ax.add_patch(r)
axis('off');

In [ ]:
TEST_IMAGES = glob('./data/scraped_from_tom/*.png')
print "Found", len(TEST_IMAGES), "images to test on."

In [ ]:
test_image = TEST_IMAGES[20]
data = imread(test_image)
imshow(data);

In [ ]:
def replace_ext(f, e):
    return os.path.splitext(f)[0] + e

In [ ]:
from skimage.transform import rescale

def split_images(image, shape):
    scale = shape[0]/float(image.shape[0])
    simg = rescale(image, (scale, scale))
    if simg.shape[1] < shape[1]:
        padding = -simg.shape[1].__mod__(shape[1])
        if len(simg.shape) == 3:
            simg = np.pad(simg, ((0,0),(padding/2,padding-padding/2),(0,0)), mode='constant')
        else:
            simg = np.pad(simg, ((0,0),(padding/2,padding-padding/2)), mode='constant')
    
    for x in linspace(0, simg.shape[1]-shape[1], int(ceil(simg.shape[1]/float(shape[1]) ))):
        yield simg[:, int(x):int(x)+shape[1]]

In [ ]:
def preprocess_image(image, bgr=False):
    h, w, c = image.shape
    result = image.transpose(2,0, 1).reshape(-1, c, h, w).astype(np.float32)
    if result.max() < 2:
        result *=255
    if bgr:
        result = result[:,::-1,:,:]
    return result

In [ ]:
from collections import namedtuple
BayesClassifierOutput = namedtuple('BayesClassifierOutput', ['labels', 'probs', 'confidences'])

#Adjust batch size to fit my GPU
net.blobs['data'].reshape(1, 3, 512, 512)

def bayes_classify_image(image, samples=4):
    
    data = preprocess_image(image, bgr=True)
    #data = np.repeat(data, 2, axis=0)
    
    outputs = np.zeros((samples, 5, 512, 512))  # count, classes, height, width

    # This assumes my GPU (K40) can hold 4 images. If it cannot, then I can 
    # still accumulate multiple images but it will require multipl ecalls to
    # net.forward
    net.blobs['data'].data[:,:,:,:] = data  # Confidence estimation requires multiple passes
        
    for i in range(samples):   
        # Process the minibatch. Due to random dropout we get slightly different
        # results each time.
        net.forward()
        outputs[i,:,:,:] = net.blobs['prob'].data
        
    avg = outputs.mean(axis=0)
    var = outputs.var(axis=0)
    labels = avg.argmax(0)
    
    return BayesClassifierOutput(labels=labels, probs =avg, confidences=var)

In [ ]:
def colorize(labels, colors):
    result = np.zeros(labels.shape + (3,), dtype=np.uint8)
    rgb = colors.values()
    indices = colors.keys()
    for i in range(len(indices)):
        mask = labels==i
        color = rgb[i]
        result[mask,0]=color[0]
        result[mask,1]=color[1]
        result[mask,2]=color[2]
    return result

In [ ]:
tiles = list(split_images(imread(TEST_IMAGES[10]), (512, 512)))
tile = tiles[1]
imshow(tile)

In [ ]:
segmentation = bayes_classify_image(tile, samples=10)
rgb = colorize(segmentation.labels, COLORS)

figure(figsize=(6, 15))
subplot(5,2,1)
imshow(tile/tile.max())
axis('off')
title('Input')

subplot(5,2,3)
imshow(rgb)
axis('off')
title('Labels(argmax)')

subplot(5,2,4)
imshow(segmentation.confidences.max(axis=0), cmap=cm.gray)
axis('off')
title('Unc(Overall)')

subplot(5,2,5)
imshow(segmentation.probs[INDEX['FacadeEdge']], cmap=cm.gray)
axis('off')
title('Pr(FacadeEdge)')

subplot(5,2,6)
imshow(segmentation.confidences[INDEX['FacadeEdge']], cmap=cm.gray)
axis('off')
title('Unc(FacadeEdge)')


subplot(5,2,7)
imshow(segmentation.probs[INDEX['Window']], cmap=cm.gray)
axis('off')
title('Pr(Window)')

subplot(5,2,8)
imshow(segmentation.confidences[INDEX['Window']], cmap=cm.gray)
axis('off')
title('Unc(Window)')

subplot(5,2,9)
imshow(segmentation.probs[INDEX['Door']], cmap=cm.gray)
axis('off')
title('Pr(Door)')

subplot(5,2,10)
imshow(segmentation.confidences[INDEX['Door']], cmap=cm.gray)
axis('off')
title('Unc(Door)')




In [ ]:
net.blobs

In [ ]:
def process_bayes(data, plot=False):
    pass

In [ ]:
from skimage.morphology import disk, binary_opening
from skimage.measure import regionprops, label

def get_boxes(image, threshold=0.5, se=disk(3)):
    mask = image>threshold
    mask = binary_opening(mask, selem=se)
    return [r.bbox for r in regionprops(label(mask))]


In [ ]:
def plot_boxes(image, boxes):
    figure(figsize=(10,5))
    axis('off')
    imshow(image, cmap=cm.gray_r)
    ax = gca()
    for (top, left, bottom, right) in boxes:
        r = Rectangle((left, top), abs(right-left), abs(top-bottom), color='red', alpha=0.5)
        ax.add_patch(r)
        
plot_boxes(data, get_boxes(process_windows_and_facades(data)[1]))

In [ ]:
def find_troughs(proj_hist, threshold = .20):
    sig = proj_hist < threshold*proj_hist.max()
    start = np.where(sig[:-1] & ~ sig[1:])[0]
    stop = np.where(~sig[:-1] & sig[1:])[0]

    if start[0] < stop[0]:
        stop = stop[1:]
    
    n = min(len(start), len(stop))
    
    mid = (start[:n]+stop[:n])/2
    
    return mid


In [ ]:
start

In [ ]:
from scipy import signal
signal.argrelextrema(window.sum(axis=0), lambda a, b: a < b)

In [ ]:
# for j, test_image in enumerate(TEST_IMAGES):
#     print "\r", j, "of", len(TEST_IMAGES),
#     process_image(test_image)
# print 

In [ ]:
def bottom_fraction(im, frac):
    return im[int((1-frac)*im.shape[0]):,:,:]

from skimage.transform import resize

def resize_image(im, w, h):
    scale = max([im.shape[1]/float(w), im.shape[0]/float(h)])
    sw = int(floor(im.shape[1]/scale))
    sh = int(floor(im.shape[0]/scale))
    sim = resize(im, (sh, sw, 3))
    result = np.zeros((h, w, 3))
    x0 = 0
    x1 = sw
    y0 = 0
    y1 = sh
    result[y0:y1, x0:x1,:] = sim
    
    if im.dtype == np.uint8:
        result= (result*0xFF).astype(np.uint8)
    return result, (sw, sh, scale)

#imshow(bottom_fraction(data, .75))
imshow(resize_image(bottom_fraction(data, .75), TILE_WIDTH, TILE_HEIGHT)[0])

In [ ]:
b2, (w, h, s) = resize_image(bottom_fraction(data, 3./4.), 480,360)
weights = classify_image(b2)
weights = softmax(weights)

fig = gcf()
fig.set_facecolor('gray')
subplot(1,2,1)
imshow(b2[:h, :w])
axis('off')
subplot(1,2,2)
imshow(weights[1][:h,:w], cmap=cm.binary_r)
axis('off');